In [1]:
import sys
from pathlib import Path

In [2]:
module_path = Path.cwd().parent
if module_path not in sys.path:
    sys.path.append(str(module_path))

In [3]:
import numpy as np
import pandas as pd

from multiprocessing import Pool
import pickle

from IPython.display import display, Markdown

In [4]:
from src.tsp_problem import TSPProblem
from src.algorithms.methods import Method
from src.visualization import Visualizer


In [12]:
problem = TSPProblem(no_runs=1)

Loaded 4 instances: TSPA, TSPB, TSPC, TSPD
Available methods: ['RANDOM_SEQUENCE', 'NEAREST', 'GREEDY_CYCLE', 'GREEDY_REGRET', 'GREEDY_REGRET_WEIGHTED', 'LOCAL_SEARCH_STEEPEST_NODES_RANDOM', 'LOCAL_SEARCH_STEEPEST_NODES_HEURISTIC', 'LOCAL_SEARCH_STEEPEST_EDGES_RANDOM', 'LOCAL_SEARCH_STEEPEST_EDGES_HEURISTIC', 'LOCAL_SEARCH_GREEDY_NODES_RANDOM', 'LOCAL_SEARCH_GREEDY_NODES_HEURISTIC', 'LOCAL_SEARCH_GREEDY_EDGES_RANDOM', 'LOCAL_SEARCH_GREEDY_EDGES_HEURISTIC', 'LOCAL_SEARCH_CANDIDATES_RANDOM', 'LOCAL_SEARCH_CANDIDATES_HEURISTIC', 'LOCAL_SEARCH_MOVES_RANDOM', 'LOCAL_SEARCH_MSLS', 'LOCAL_SEARCH_ITERATIVE', 'LOCAL_SEARCH_LSN_NO_LS', 'LOCAL_SEARCH_LSN_WITH_LS']


In [13]:
items = items = [
    (("TSPC",), [Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM]),
    (("TSPC",), [Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM]),
    (("TSPC",), [Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM]),
    (("TSPC",), [Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM]),
    (("TSPC",), [Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM]),
    (("TSPC",), [Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM]),
    (("TSPC",), [Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM]),
    (("TSPC",), [Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM]),
    (("TSPC",), [Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM]),
    (("TSPC",), [Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM]),
]

In [14]:
with Pool() as pool:
    grades_tmp = pool.starmap(problem.run, items)


Running TSPC instance
Running TSPC instance

Running TSPC instance
Running TSPC instance
Running TSPC instance


Running TSPC instance


Running TSPC instance

Running LOCAL_SEARCH_GREEDY_EDGES_RANDOM method for 1 runs
Running TSPC instanceRunning LOCAL_SEARCH_GREEDY_EDGES_RANDOM method for 1 runs
Running LOCAL_SEARCH_GREEDY_EDGES_RANDOM method for 1 runsRunning LOCAL_SEARCH_GREEDY_EDGES_RANDOM method for 1 runs
Running LOCAL_SEARCH_GREEDY_EDGES_RANDOM method for 1 runs

  1 /   1Running LOCAL_SEARCH_GREEDY_EDGES_RANDOM method for 1 runs

  1 /   1Running LOCAL_SEARCH_GREEDY_EDGES_RANDOM method for 1 runs

  1 /   1Running LOCAL_SEARCH_GREEDY_EDGES_RANDOM method for 1 runs
Finished LOCAL_SEARCH_GREEDY_EDGES_RANDOM method in 62.11s

Running TSPC instance
Running LOCAL_SEARCH_GREEDY_EDGES_RANDOM method for 1 runs
Finished LOCAL_SEARCH_GREEDY_EDGES_RANDOM method in 66.30s

Running TSPC instance
Running LOCAL_SEARCH_GREEDY_EDGES_RANDOM method for 1 runs
Finished LOCAL_SEARCH_GREEDY_EDGES_

In [18]:
pickle.dump(grades_tmp, open("grades_tmp.pkl", "wb"))

In [ ]:
print("Done!")